# 🔍 SelfQueryRetriever - 똑똑한 검색의 마법사

## 📚 개요

**SelfQueryRetriever** 는 마치 **똑똑한 도서관 사서**처럼 작동하는 특별한 검색 도구입니다! 🧙‍♂️

### 🎯 일반적인 검색 vs SelfQueryRetriever

#### 🔍 **일반적인 검색**
- 사용자: "2023년도 스킨케어 제품 추천해줘"
- 시스템: "2023", "스킨케어", "추천" 키워드로만 검색 😵

#### ⚡ **SelfQueryRetriever의 검색**
- 사용자: "2023년도 스킨케어 제품 추천해줘"
- 시스템: 
  - "아! 연도는 2023이고"
  - "카테고리는 스킨케어네!"
  - "메타데이터 필터를 적용해서 정확히 찾아볼게!" 🎯

### 🏪 쇼핑몰 직원으로 이해하기

**일반 직원**: "스킨케어요? 매장 전체를 둘러보세요!"  
**베테랑 직원 (SelfQueryRetriever)**: "2023년도 스킨케어 제품이시군요! 3층 A구역에 있습니다. 평점 높은 순으로 보여드릴까요?"

### 🚀 핵심 장점

✅ **정확한 필터링**: 메타데이터를 활용한 정밀 검색  
✅ **자연어 이해**: 복잡한 조건도 자연어로 표현  
✅ **유연한 조건**: "4.5점 이상", "2023년도", "메이크업" 등 다양한 조건  
✅ **똑똑한 해석**: 사용자 의도를 정확하게 파악

## 📋 목차

1. **🛠️ 환경 설정** - 튜토리얼 시작 준비
2. **📦 샘플 데이터 생성** - 화장품 데이터로 실습
3. **🔧 SelfQueryRetriever 구성** - 핵심 기능 설정
4. **🎯 기본 검색 테스트** - 단순 조건 검색
5. **⚡ 복합 조건 검색** - 여러 조건 동시 적용
6. **🎛️ 검색 결과 수 제한** - k 파라미터 활용
7. **🔬 내부 동작 원리** - 깊이 있는 이해

### 💡 이 튜토리얼에서 배울 것들

- SelfQueryRetriever의 동작 원리와 장점
- 메타데이터를 활용한 정밀 검색 구현
- 자연어 쿼리를 구조화된 검색으로 변환
- 복합 조건 검색과 결과 제한 기법
- Query Constructor와 Translator의 역할

---

In [ ]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv(override=True)

## 🛠️ 환경 설정

SelfQueryRetriever의 강력한 기능을 체험하기 전에 필요한 도구들을 준비해봅시다!

In [ ]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("LangChain-Tutorial")

---

## 📦 샘플 데이터 생성

### 🏪 화장품 쇼핑몰 데이터베이스 만들기

실제 화장품 온라인몰에서 사용할 법한 상품 데이터를 만들어보겠습니다!

#### 📊 우리가 만들 데이터 구조

각 화장품마다 다음 정보들을 포함합니다:

- **📝 상품 설명**: 제품의 특징과 효과
- **📅 출시 연도**: 언제 출시되었는지
- **🏷️ 카테고리**: 스킨케어, 메이크업, 클렌징, 선케어 등
- **⭐ 사용자 평점**: 1~5점 척도의 만족도

### 🔧 벡터 저장소 구축하기

화장품 상품 정보를 **벡터 형태로 저장**하여 의미적 유사도 검색이 가능한 데이터베이스를 만들어봅시다.

#### 💡 벡터 저장소란?
텍스트를 **숫자 벡터로 변환**하여 저장하는 특별한 데이터베이스입니다. 마치 **단어의 DNA**를 분석해서 비슷한 의미의 제품을 찾아주는 똑똑한 시스템이죠!

In [ ]:
from langchain_chroma import Chroma
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings

# 화장품 상품의 설명과 메타데이터 생성
docs = [
    Document(
        page_content="수분 가득한 히알루론산 세럼으로 피부 속 깊은 곳까지 수분을 공급합니다.",
        metadata={"year": 2024, "category": "스킨케어", "user_rating": 4.7},  # 메타데이터: 연도, 카테고리, 평점
    ),
    Document(
        page_content="24시간 지속되는 매트한 피니시의 파운데이션, 모공을 커버하고 자연스러운 피부 표현이 가능합니다.",
        metadata={"year": 2023, "category": "메이크업", "user_rating": 4.5},  # 메타데이터: 연도, 카테고리, 평점
    ),
    Document(
        page_content="식물성 성분으로 만든 저자극 클렌징 오일, 메이크업과 노폐물을 부드럽게 제거합니다.",
        metadata={"year": 2023, "category": "클렌징", "user_rating": 4.8},  # 메타데이터: 연도, 카테고리, 평점
    ),
    Document(
        page_content="비타민 C 함유 브라이트닝 크림, 칙칙한 피부톤을 환하게 밝혀줍니다.",
        metadata={"year": 2023, "category": "스킨케어", "user_rating": 4.6},  # 메타데이터: 연도, 카테고리, 평점
    ),
    Document(
        page_content="롱래스팅 립스틱, 선명한 발색과 촉촉한 사용감으로 하루종일 편안하게 사용 가능합니다.",
        metadata={"year": 2024, "category": "메이크업", "user_rating": 4.4},  # 메타데이터: 연도, 카테고리, 평점
    ),
    Document(
        page_content="자외선 차단 기능이 있는 톤업 선크림, SPF50+/PA++++ 높은 자외선 차단 지수로 피부를 보호합니다.",
        metadata={"year": 2024, "category": "선케어", "user_rating": 4.9},  # 메타데이터: 연도, 카테고리, 평점
    ),
]

# 벡터 저장소 생성: 문서들을 벡터로 변환하여 Chroma DB에 저장
vectorstore = Chroma.from_documents(
    docs, OpenAIEmbeddings(model="text-embedding-3-small")  # OpenAI 임베딩 모델 사용
)

---

## 🔧 SelfQueryRetriever 구성

### 🎯 똑똑한 검색 시스템 만들기

이제 우리의 **SelfQueryRetriever** 를 만들어봅시다! 이 시스템이 제대로 작동하려면 두 가지 핵심 정보가 필요합니다:

#### 1️⃣ **메타데이터 필드 정보** 📊
- 어떤 종류의 필터가 가능한지 알려주는 **설계도**
- 예: "카테고리는 문자열, 연도는 숫자, 평점은 실수"

#### 2️⃣ **문서 내용 설명** 📝  
- 저장된 문서들이 어떤 내용인지에 대한 **간단한 요약**
- 예: "화장품 상품 정보"

### 🔍 왜 이 정보들이 필요할까?

마치 **새로운 직원에게 업무를 가르치는 것**과 같습니다:

- **"우리 매장에는 어떤 상품들이 있나요?"** → 문서 내용 설명
- **"어떤 방식으로 분류되어 있나요?"** → 메타데이터 필드 정보

이 정보가 있어야 AI가 사용자의 질문을 정확히 이해하고 적절한 검색을 수행할 수 있습니다! 🎯

### 📋 메타데이터 필드 정보 정의하기

`AttributeInfo` 는 각 메타데이터 필드의 **신상명세서**라고 생각하면 됩니다! 

#### 🏷️ 우리 화장품 데이터의 신상명세서

각 필드마다 다음 정보를 정의합니다:

##### 1️⃣ **카테고리 (category)**
- **타입**: 문자열 (string) 📝
- **설명**: 화장품의 종류
- **가능한 값**: 스킨케어, 메이크업, 클렌징, 선케어

##### 2️⃣ **출시 연도 (year)**  
- **타입**: 정수 (integer) 🔢
- **설명**: 제품이 출시된 연도
- **예시**: 2023, 2024

##### 3️⃣ **사용자 평점 (user_rating)**
- **타입**: 실수 (float) ⭐
- **설명**: 고객 만족도 점수
- **범위**: 1.0 ~ 5.0점

### 💡 왜 이렇게 정의할까?

AI가 **"평점이 4.5 이상인 제품"** 이라는 요청을 받으면:
- "아! user_rating이 4.5보다 큰 제품을 찾아야겠군!" 🎯
- 정확한 필터 조건으로 변환해서 검색 수행

In [ ]:
from langchain.chains.query_constructor.base import AttributeInfo


# 메타데이터 필드 정보 생성: AI가 각 필드를 어떻게 해석해야 하는지 알려주는 정보
metadata_field_info = [
    AttributeInfo(
        name="category",  # 필드명: category
        description="The category of the cosmetic product. One of ['스킨케어', '메이크업', '클렌징', '선케어']",  # 화장품 카테고리 설명
        type="string",  # 데이터 타입: 문자열
    ),
    AttributeInfo(
        name="year",  # 필드명: year
        description="The year the cosmetic product was released",  # 제품 출시 연도 설명
        type="integer",  # 데이터 타입: 정수
    ),
    AttributeInfo(
        name="user_rating",  # 필드명: user_rating
        description="A user rating for the cosmetic product, ranging from 1 to 5",  # 사용자 평점 설명 (1-5점)
        type="float",  # 데이터 타입: 실수
    ),
]

### ⚡ SelfQueryRetriever 객체 생성하기

이제 모든 준비가 끝났습니다! **똑똑한 검색 시스템**을 조립해봅시다! 🚀

#### 🧩 필요한 구성 요소들

- **🧠 llm**: 사용자 질문을 이해하는 언어 모델
- **🗄️ vectorstore**: 상품 정보가 저장된 벡터 데이터베이스  
- **📝 document_contents**: 문서 내용에 대한 간단한 설명
- **📊 metadata_field_info**: 메타데이터 필드 정보 (위에서 정의한 것!)

#### 🎭 SelfQueryRetriever의 동작 과정

1. **👂 질문 듣기**: "2023년도 스킨케어 제품 추천해줘"
2. **🧠 질문 분석**: LLM이 질문을 이해하고 구조화
3. **🔍 필터 생성**: year=2023, category="스킨케어"
4. **📦 검색 실행**: 조건에 맞는 상품들을 벡터스토어에서 검색
5. **🎁 결과 반환**: 찾은 상품들을 사용자에게 전달

In [ ]:
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_openai import ChatOpenAI

# LLM 정의: 사용자 질문을 이해하고 구조화된 쿼리를 생성할 언어 모델
llm = ChatOpenAI(model="gpt-4.1", temperature=0)

# SelfQueryRetriever 생성: 자연어 질문을 메타데이터 필터가 포함된 검색으로 변환
retriever = SelfQueryRetriever.from_llm(
    llm=llm,  # 사용할 언어 모델
    vectorstore=vectorstore,  # 검색할 벡터 저장소
    document_contents="Brief summary of a cosmetic product",  # 문서 내용에 대한 간단한 설명
    metadata_field_info=metadata_field_info,  # 메타데이터 필드 정보
)

---

## 🎯 기본 검색 테스트

### 🔍 단일 조건 검색해보기

이제 우리의 **SelfQueryRetriever** 가 얼마나 똑똑한지 테스트해봅시다! 

다양한 조건으로 검색을 해보면서 어떻게 자동으로 적절한 필터를 생성하는지 확인해보겠습니다. ✨

#### 🎮 테스트 시나리오

각 검색에서 **SelfQueryRetriever** 가 어떻게 똑똑하게 필터를 적용하는지 확인해봅시다!

##### 1️⃣ **평점 조건 검색** ⭐
"평점이 4.8 이상인 제품" → `user_rating >= 4.8` 필터 자동 생성

##### 2️⃣ **연도 조건 검색** 📅  
"2023년에 출시된 상품" → `year = 2023` 필터 자동 생성

##### 3️⃣ **카테고리 조건 검색** 🏷️
"선케어 상품" → `category = "선케어"` 필터 자동 생성

In [ ]:
# Self-query 검색: 평점 조건 필터링
# "평점이 4.8 이상인 제품" → user_rating >= 4.8 조건으로 자동 변환
retriever.invoke("평점이 4.8 이상인 제품을 추천해주세요")

In [ ]:
# Self-query 검색: 연도 조건 필터링  
# "2023년에 출시된 상품" → year = 2023 조건으로 자동 변환
retriever.invoke("2023년에 출시된 상품을 추천해주세요")

In [ ]:
# Self-query 검색: 카테고리 조건 필터링
# "카테고리가 선케어인 상품" → category = "선케어" 조건으로 자동 변환
retriever.invoke("카테고리가 선케어인 상품을 추천해주세요")

---

## ⚡ 복합 조건 검색

### 🎯 여러 조건을 동시에 적용하기

**SelfQueryRetriever** 의 진짜 실력은 **복합 조건 검색**에서 나타납니다! 

#### 🔍 복합 조건 검색의 위력

**일반 검색**: "메이크업 4.5점"이라는 키워드로만 검색 😵  
**SelfQueryRetriever**: 
- 카테고리 = "메이크업" **AND** 
- 평점 >= 4.5 
- 이 두 조건을 **동시에 만족하는** 제품만 검색! 🎯

### 💡 실생활 쇼핑 시나리오

마치 쇼핑몰에서 **"브랜드: 샤넬, 가격: 10만원 이하, 평점: 4점 이상"** 처럼 여러 필터를 동시에 적용하는 것과 같습니다!

In [ ]:
# Self-query 검색: 복합 조건 필터링 (카테고리 + 평점)
# "메이크업 + 평점 4.5 이상" → category = "메이크업" AND user_rating >= 4.5 조건으로 자동 변환
retriever.invoke(
    "카테고리가 메이크업인 상품 중에서 평점이 4.5 이상인 상품을 추천해주세요"
)

---

## 🎛️ 검색 결과 수 제한하기

### 📊 k 파라미터의 마법

때로는 **"상위 3개만 보여줘"** 처럼 검색 결과 개수를 제한하고 싶을 때가 있죠!

#### 🔢 k 파라미터란?
- **k**: "몇 개의 결과를 가져올까?"를 결정하는 숫자
- **기본값**: 보통 4개 정도
- **사용법**: `enable_limit=True` + `search_kwargs={"k": 원하는_개수}`

### 🎯 두 가지 제한 방법

#### 1️⃣ **코드로 제한** 💻
```python
search_kwargs={"k": 2}  # 항상 2개만 가져와줘!
```

#### 2️⃣ **자연어로 제한** 🗣️
```python
"2023년 상품 3개만 추천해줘"  # AI가 알아서 3개로 제한!
```

### 💡 언제 사용할까?

✅ **빠른 미리보기**: 대략적인 결과만 확인하고 싶을 때  
✅ **성능 최적화**: 너무 많은 결과로 느려지는 것을 방지  
✅ **UI 제약**: 화면에 표시할 공간이 제한적일 때

In [ ]:
# 검색 결과 수 제한 기능이 있는 retriever 생성
retriever = SelfQueryRetriever.from_llm(
    llm=llm,  # 언어 모델
    vectorstore=vectorstore,  # 벡터 저장소
    document_contents="Brief summary of a cosmetic product",  # 문서 내용 설명
    metadata_field_info=metadata_field_info,  # 메타데이터 필드 정보
    enable_limit=True,  # 검색 결과 제한 기능을 활성화
    search_kwargs={"k": 2},  # k 값을 2로 지정하여 검색 결과를 2개로 제한
)

#### 🧪 실험해보기

우리 데이터에는 **2023년 출시 상품이 3개**가 있지만, k=2로 제한해서 **상위 2개만** 가져와보겠습니다!

In [ ]:
# Self-query 검색: k=2로 제한된 결과
# 2023년 출시 상품은 3개 있지만 k=2 설정으로 상위 2개만 반환
retriever.invoke("2023년에 출시된 상품을 추천해주세요")

### 🗣️ 자연어로 개수 지정하기

**더욱 직관적인 방법**이 있습니다! 코드에서 `search_kwargs`를 설정하지 않고도, **질문 안에 숫자를 포함시키면** AI가 자동으로 인식합니다! 

#### ✨ 자연어 개수 제한의 마법

- **"1개만 추천해줘"** → k=1 자동 설정
- **"상위 3개 보여줘"** → k=3 자동 설정  
- **"5개 정도만"** → k=5 자동 설정

이렇게 하면 사용자가 **더 자연스럽게** 요청할 수 있어서 실제 서비스에서 매우 유용합니다! 🚀

In [ ]:
# 자연어로 개수 제한이 가능한 retriever 생성 (search_kwargs 없음)
retriever = SelfQueryRetriever.from_llm(
    llm=llm,  # 언어 모델
    vectorstore=vectorstore,  # 벡터 저장소  
    document_contents="Brief summary of a cosmetic product",  # 문서 내용 설명
    metadata_field_info=metadata_field_info,  # 메타데이터 필드 정보
    enable_limit=True,  # 검색 결과 제한 기능을 활성화 (자연어 개수 인식 가능)
)

# Self-query 검색: 자연어로 개수 지정
# "1개를 추천해주세요" → AI가 자동으로 k=1 설정
retriever.invoke("2023년에 출시된 상품 1개를 추천해주세요")

In [ ]:
# Self-query 검색: 자연어로 개수 지정 (2개)
# "2개를 추천해주세요" → AI가 자동으로 k=2 설정
retriever.invoke("2023년에 출시된 상품 2개를 추천해주세요")

---

## 🔬 내부 동작 원리 탐구

### 🕵️‍♂️ SelfQueryRetriever 속으로!

지금까지 **"마법 같은 검색"** 을 경험해봤는데, 궁금하지 않으신가요? **내부에서는 도대체 무슨 일이 일어나는 걸까요?** 🤔

#### 🎭 SelfQueryRetriever의 내부 구조

**SelfQueryRetriever** 는 실제로는 **두 개의 핵심 컴포넌트**로 구성되어 있습니다:

##### 1️⃣ **Query Constructor (쿼리 생성기)** 🏗️
- **역할**: 자연어 질문을 구조화된 쿼리로 변환
- **예시**: "2023년 스킨케어" → `{filter: {year: 2023, category: "스킨케어"}}`

##### 2️⃣ **Structured Query Translator (쿼리 변환기)** 🔄
- **역할**: 구조화된 쿼리를 벡터스토어용 필터로 변환
- **예시**: `{year: 2023}` → Chroma 데이터베이스 필터 문법

### 🎯 왜 이렇게 나누어져 있을까?

마치 **번역가와 통역사**가 협력하는 것처럼:

- **Query Constructor**: "한국어 → 국제 공용어" 번역가
- **Translator**: "국제 공용어 → 각 나라 방언" 통역사

이렇게 분리하면 **다양한 벡터스토어**(Chroma, Pinecone, Weaviate 등)에서 재사용이 가능합니다! 🚀

### 🔧 직접 만들어보는 이유

`SelfQueryRetriever.from_llm()` 은 편리하지만, **더 세밀한 제어**가 필요할 때는 직접 조립하는 것이 좋습니다:

✅ **프롬프트 커스터마이징**: 우리 도메인에 맞는 프롬프트 작성  
✅ **에러 핸들링**: 예외 상황 대응 로직 추가  
✅ **성능 최적화**: 불필요한 단계 제거  
✅ **디버깅**: 각 단계별 결과 확인 가능

### 📚 참고 자료
고급 활용법이 궁금하시다면 [LangChain 공식 쿡북](https://github.com/langchain-ai/langchain/blob/master/cookbook/self_query_hotel_search.ipynb)을 참고해보세요!

### 🏗️ Query Constructor Chain 생성하기

**Query Constructor** 는 SelfQueryRetriever의 **핵심 두뇌**입니다! 사용자의 자연어 질문을 받아서 구조화된 검색 조건으로 바꿔주는 역할을 합니다.

#### 🧩 필요한 구성 요소들

##### 1️⃣ **Prompt (프롬프트)** 📝
- AI에게 "이렇게 이해해줘"라고 알려주는 **지시서**
- 메타데이터 정보와 예시를 포함
- `get_query_constructor_prompt()` 함수로 자동 생성

##### 2️⃣ **Output Parser (출력 파서)** 🔄
- AI의 응답을 **구조화된 형태로 변환**하는 도구  
- JSON 형태를 Python 객체로 변환
- `StructuredQueryOutputParser` 사용

##### 3️⃣ **LLM (언어 모델)** 🧠
- 실제로 질문을 **이해하고 분석**하는 AI
- GPT-4.1 같은 대화형 AI 모델

### 📝 쿼리 생성기 프롬프트 만들기

`get_query_constructor_prompt()` 함수는 **AI를 위한 완벽한 매뉴얼**을 자동으로 생성해줍니다!

#### 🎯 프롬프트에 포함되는 내용들

- **📊 메타데이터 필드 설명**: 어떤 필터들이 가능한지
- **📋 문서 내용 요약**: 무엇에 대한 데이터인지  
- **📚 예시 질문들**: "이런 질문이 들어오면 이렇게 해석해줘"
- **🔧 출력 형식**: 결과를 어떤 구조로 만들어야 하는지

#### 💡 프롬프트의 역할

마치 **신입사원에게 업무 가이드를 주는 것**과 같습니다:
- "우리 회사 상품은 이런 것들이야"
- "고객이 이렇게 물어보면 저렇게 대답해줘"  
- "결과는 이런 형식으로 정리해서 보고해줘"

In [ ]:
from langchain.chains.query_constructor.base import (
    StructuredQueryOutputParser,
    get_query_constructor_prompt,
)

# 문서 내용 설명과 메타데이터 필드 정보를 사용하여 쿼리 생성기 프롬프트를 가져옵니다.
prompt = get_query_constructor_prompt(
    "Brief summary of a cosmetic product",  # 문서 내용 설명
    metadata_field_info,  # 메타데이터 필드 정보 (위에서 정의한 AttributeInfo 리스트)
)

# StructuredQueryOutputParser 를 생성: AI 응답을 구조화된 쿼리 객체로 변환
output_parser = StructuredQueryOutputParser.from_components()

# query_constructor chain 을 생성: prompt → llm → output_parser 순으로 연결
query_constructor = prompt | llm | output_parser

#### 🔍 생성된 프롬프트 내용 확인하기

**AI에게 어떤 지시를 내리고 있는지** 직접 확인해볼까요? `prompt.format()` 메서드로 실제 프롬프트 내용을 출력해보겠습니다.

이렇게 하면 **AI가 우리 데이터를 어떻게 이해하고 있는지**, 그리고 **어떤 방식으로 질문을 해석하려고 하는지** 알 수 있어서 매우 유용합니다! 📋

In [ ]:
# 프롬프트 내용 출력: AI에게 어떤 지시사항을 주고 있는지 확인
# "dummy question"을 매개변수로 하여 실제 프롬프트 템플릿을 확인
print(prompt.format(query="dummy question"))

#### ⚡ Query Constructor 실행해보기

이제 **복잡한 질문**을 실제로 Query Constructor에 넣어보면서 **어떻게 구조화된 쿼리로 변환되는지** 확인해봅시다!

##### 🎯 테스트 질문
**"2023년도에 출시한 상품 중 평점이 4.5 이상인 상품중에서 스킨케어 제품을 추천해주세요"**

이 복잡한 조건들이 어떻게 정확한 필터 조건으로 변환되는지 지켜보세요! ✨

In [ ]:
# Query Constructor 실행: 복잡한 자연어 질문을 구조화된 쿼리로 변환
query_output = query_constructor.invoke(
    {
        # 복잡한 조건이 포함된 질문: 연도(2023) + 평점(4.5이상) + 카테고리(스킨케어)  
        "query": "2023년도에 출시한 상품 중 평점이 4.5 이상인 상품중에서 스킨케어 제품을 추천해주세요"
    }
)

#### 🔎 생성된 구조화 쿼리 분석하기

자! 이제 AI가 우리 질문을 **어떻게 해석했는지** 확인해봅시다! 

**원본 질문**: "2023년도에 출시한 상품 중 평점이 4.5 이상인 상품중에서 스킨케어 제품을 추천해주세요"

이 질문에서 AI가 추출해낸 **필터 조건들**을 살펴보겠습니다. 👇

In [ ]:
# 생성된 구조화 쿼리의 필터 조건 확인
# AI가 자연어 질문에서 추출한 필터 조건들을 출력
query_output.filter.arguments

### 🎯 Query Constructor 최적화의 중요성

**SelfQueryRetriever의 성능은 Query Constructor가 좌우합니다!** 마치 **통역사의 실력이 국제회의의 성공을 결정하는 것**과 같아요.

#### 🔧 최적화 방법들

##### 1️⃣ **프롬프트 튜닝** 📝
- 우리 도메인에 특화된 예시 추가
- 더 명확한 지시사항 작성
- 자주 발생하는 실수 패턴 방지

##### 2️⃣ **속성 설명 개선** 📊  
- 메타데이터 필드의 의미를 더 명확히 설명
- 가능한 값들의 예시 제공
- 데이터 타입과 범위 정확히 명시

##### 3️⃣ **예시 추가** 📚
- 프롬프트 내에 Few-shot 예시 추가
- 복잡한 질문 패턴에 대한 답변 예시
- 경계 케이스 처리 방법 안내

### 💡 실전 팁

좋은 SelfQueryRetriever를 만들려면:

✅ **도메인 전문성**: 해당 분야의 전문 용어와 패턴 이해  
✅ **사용자 행동 분석**: 실제 사용자들이 어떻게 질문하는지 파악  
✅ **반복 테스트**: 다양한 질문으로 지속적으로 성능 검증  
✅ **점진적 개선**: 실패 사례를 분석해서 프롬프트 개선

**결국 Query Constructor는 우리가 얼마나 잘 가르쳐주느냐에 따라 성능이 결정됩니다!** 🎯

---

### 🔄 구조화된 쿼리 변환기 (Structured Query Translator)

### 🌐 각 벡터스토어 전용 통역사

**Structured Query Translator** 는 **각 벡터스토어의 전용 통역사**입니다! 

#### 🗣️ 왜 통역사가 필요할까?

같은 의미라도 **각 벡터스토어마다 문법이 다르기 때문**입니다:

##### 📊 예시: "2023년 데이터" 필터링

**공통 구조화 쿼리**: `{year: 2023}`

**각 벡터스토어별 문법**:
- **Chroma**: `{"year": {"$eq": 2023}}`
- **Pinecone**: `year=2023`  
- **Weaviate**: `where: {path: ["year"], operator: Equal, valueInt: 2023}`

#### 🎯 ChromaTranslator의 역할

우리는 **Chroma 데이터베이스**를 사용하므로 `ChromaTranslator`가 필요합니다:

- **입력**: 표준 구조화 쿼리
- **출력**: Chroma 전용 필터 문법
- **역할**: "Chroma야, 이런 조건으로 검색해줘!"

### 💡 확장성의 장점

이런 구조 덕분에:
- **새로운 벡터스토어** 추가시 Translator만 바꾸면 OK! 
- **Query Constructor는 재사용** 가능
- **일관된 인터페이스** 제공

In [ ]:
from langchain.retrievers.self_query.chroma import ChromaTranslator

# 직접 구성한 구성 요소들로 SelfQueryRetriever 생성
retriever = SelfQueryRetriever(
    query_constructor=query_constructor,  # 이전에 생성한 query_constructor chain을 지정 (자연어 → 구조화 쿼리)
    vectorstore=vectorstore,  # 검색할 벡터 저장소 (Chroma DB)
    structured_query_translator=ChromaTranslator(),  # 쿼리 변환기 (구조화 쿼리 → Chroma 필터 문법)
)

### 🎉 완성된 SelfQueryRetriever 테스트

드디어 **직접 만든 SelfQueryRetriever** 가 완성되었습니다! 이제 같은 복잡한 질문으로 테스트해보면서 `from_llm()` 으로 만든 것과 동일하게 작동하는지 확인해봅시다.

#### 🔍 최종 테스트 질문
**"2023년도에 출시한 상품 중 평점이 4.5 이상인 상품중에서 스킨케어 제품을 추천해주세요"**

이 질문이 다음 과정을 거쳐 처리됩니다:
1. **Query Constructor**: 자연어 → 구조화 쿼리
2. **ChromaTranslator**: 구조화 쿼리 → Chroma 필터
3. **Vectorstore**: 필터링된 검색 실행
4. **결과 반환**: 조건에 맞는 상품들

In [ ]:
retriever.invoke(
    # 질문
    "2023년도에 출시한 상품 중 평점이 4.5 이상인 상품중에서 스킨케어 제품을 추천해주세요"
)